In [1]:

import metrics_calc
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer


c:\Users\97254\Desktop\niv\Github projects\NLP-Final-Project---Dreams-Interpreter\dreams_interpreter\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.cuda.get_device_name(0))  # Should show your GPU name

# Check if GPU (CUDA) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load trained model and tokenizer
model_path = "./dream_model_gpt2"
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)  # Move model to GPU if available
tokenizer = GPT2Tokenizer.from_pretrained(model_path)



2.6.0+cu118
True
NVIDIA GeForce MX450
Using device: cuda


In [7]:

# # Read validation dataset
# input_csv = "DREAMS DATA\dreams_and_interpretations_Freud.xlsx"  # Replace with your actual CSV file name
# df = pd.read_csv(input_csv)

# Read validation dataset (Excel file)
input_excel = r"C:\\Users\97254\Desktop\\niv\\Github projects\\NLP-Final-Project---Dreams-Interpreter\\DREAMS DATA\\dreams_and_interpretations_Freud.xlsx"  # Replace with your actual Excel file name
df = pd.read_excel(input_excel)


# Ensure the dataset has a 'Dream' column
if 'Dream' not in df.columns:
    raise ValueError("CSV file must contain a 'Dream' column.")

# Function to generate interpretations
def generate_interpretation(dream):
    input_text = f"Dream: {dream}\nInterpretation:"
    # input_ids = tokenizer.encode(input_text, return_tensors="pt")
    
     # Tokenize and move input tensors to the correct device (GPU if available)
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate text
    # output_ids = model.generate(input_ids, max_length=100, num_return_sequences=1,  repetition_penalty=1.2, pad_token_id=tokenizer.eos_token_id)
    output_ids = model.generate(
    input_ids, 
    max_length=70,  
    num_return_sequences=1, 
    repetition_penalty=1.2,  
    # do_sample=True,  # Prevents early stopping
    pad_token_id=tokenizer.eos_token_id
    )
    
    interpretation = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract generated interpretation
    return interpretation.split("Interpretation:")[-1].strip()



# Generate interpretations
df["Generated_Interpretation"] = df["Dream"].apply(generate_interpretation)

# Save to new CSV
output_csv = "C:\\Users\\97254\\Desktop\\niv\\Github projects\\NLP-Final-Project---Dreams-Interpreter\\Niv- gpt2\\gpt2_outputs\\dreams_with_generated_interpretations_gpt2.csv"
df.to_csv(output_csv, index=False)

print(f"Generated interpretations saved to {output_csv}")


Generated interpretations saved to C:\Users\97254\Desktop\niv\Github projects\NLP-Final-Project---Dreams-Interpreter\Niv- gpt2\gpt2_outputs\dreams_with_generated_interpretations_gpt2.csv


In [8]:
output_metrics_csv = "C:\\Users\\97254\\Desktop\\niv\\Github projects\\NLP-Final-Project---Dreams-Interpreter\\Niv- gpt2\\gpt2_outputs\\gpt2_metrics_results.csv"
metrics_calc.evaluate_text_metrics(tokenizer, model, output_csv, output_metrics_csv)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Rob

Metrics saved to 'C:\Users\97254\Desktop\niv\Github projects\NLP-Final-Project---Dreams-Interpreter\Niv- gpt2\gpt2_outputs\gpt2_metrics_results.csv'
